# CEL -- Common Expression Language

## Agenda

- Why CEL?

- About CEL

- Processing

- Implementation Details

## Why CEL?

https://github.com/cloud-custodian/cloud-custodian/issues/5759

## About CEL

- Keep it small & fast.

- Make it extensible.

- Developer-friendly.  Similar to C/C++/Java/JavaScript.

## Processing

1. Text of the expression. ``355. / 113.``

2. Abstract Syntax Tree. Created by a `lark` parser.  ``Tree('expr', [Tree('literal', [Token]), Tree('literal', [Token])])``.

3. Compiler to produce an executable form. This is Python. Nothing much to do here. 

4. Context with variables. None for this example.

5. Evaluation to apply executable expression to variables to get a response.

In [6]:
import celpy
env = celpy.Environment()
ast = env.compile("355. / 113.")
prgm = env.program(ast)
prgm.evaluate({})

DoubleType(3.1415929203539825)

## Implementation

- Lark used to do two things: Lexical Scanning and Parsing.

  -  Lexical Scanning locates language tokens: int, float, identifier, operator, etc.
  
  -  Parsing recognizes higher-level (possibly recursive) constructs.
  
- An "evaluator" is a Lark ``Interpreter`` subclass.

  -  Operators mapped to functions with possible run-time overrides.

- Sits on top of ``celtypes`` module with Python implementations of the CEL data types.

  -  Provides GO-like semantics

## Lark EBNF Rules

```
expr           : conditionalor ["?" conditionalor ":" expr]

conditionalor  : [conditionalor "||"] conditionaland

conditionaland : [conditionaland "&&"] relation

relation       : [relation_lt | relation_le | relation_ge | relation_gt | relation_eq | relation_ne | relation_in] addition
```

## Lark EBNF Tokens

```
INT_LIT        : /-?/ /0x/ HEXDIGIT+ | /-?/ DIGIT+

UINT_LIT       : INT_LIT /[uU]/
```

## Rule Implementation

```
    func = self.functions["_?_:_"]
    cond_value, left, right = cast(Tuple[Result, Result, Result], self.visit_children(tree))
    try:
        return func(cond_value, left, right)
    except TypeError as ex:
        logger.debug(f"{func.__name__}({left}, {right}) --> {ex}")
        err = (
            f"found no matching overload for _?_:_ "
            f"applied to '({type(cond_value)}, {type(left)}, {type(right)})'"
        )
        value = CELEvalError(err, ex.__class__, ex.args, tree=tree)
        value.__cause__ = ex
        return value
```

## celtypes

A lot of this.

```
class DoubleType(float):
    def __truediv__(self, other: Any) -> 'DoubleType':
    if cast(float, other) == 0.0:
        return DoubleType("inf")
    else:
        return DoubleType(super().__truediv__(other))

```